In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-11"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11391,2024-04-11,Noruega Blno,13:30,Fyllingen,Bærum,1.07,7.49,171.5,1.85,1.85,0.0,0.00,0.00,OxdEF1kB,0.934579,0.133511,0.540541,0.540541,0.068091,111.506,21.676588,0.194398,2.4,1.341641,0.559017,126.90,1.212,0.144983,0.119623,71.0,332.446,187.697621,0.564596,2.4,1.341641,0.559017,258.00,3.714,1.852790,0.498866,14.0,94,75,1.35,3.44,794.954,277.176,1.060660,0.000000,NaN,27.17,5.434,0.012882,0.000000,0.0,0.000000,10.26,2.052,3.162768,0.478688,0.7,0.221312
11392,2024-04-11,Reino Unido Bbl,15:45,Newcastle Eagles,Bristol Flyers,1.61,2.22,172.5,1.85,1.85,0.0,0.00,0.00,GUPuymw6,0.621118,0.450450,0.540541,0.540541,0.071568,166.658,52.797505,0.316802,1.2,1.643168,1.369306,240.00,1.786,0.435982,0.244111,22.0,246.704,93.641525,0.379570,1.2,1.643168,1.369306,305.30,3.080,1.204927,0.391210,-18.0,96,71,2.50,4.30,143.762,231.632,0.225240,0.000000,NaN,-0.78,-0.156,-3.910256,0.715491,0.5,-0.215491,1.57,0.314,3.885350,0.419383,0.3,-0.119383
11393,2024-04-11,Polônia Liga De Basquete,12:30,Gdynia,Dziki Warszawa,1.72,2.05,165.5,1.81,1.85,-2.5,1.97,2.45,Qk6gR2AP,0.581395,0.487805,0.552486,0.540541,0.069200,255.056,108.951826,0.427168,1.8,1.643168,0.912871,180.18,3.298,1.786343,0.541644,-15.0,181.800,77.074615,0.423953,0.0,0.000000,NaN,92.11,2.512,0.908746,0.361762,-95.0,77,61,2.34,1.51,361.878,244.664,0.123791,0.015456,0.153580,6.45,1.290,0.558140,0.395745,0.5,0.104255,4.14,0.828,1.268116,0.352213,0.4,0.047787
11394,2024-04-11,Nova Zelândia Nbl,04:30,Manawatu Jets,Bay Hawks,2.59,1.46,176.5,1.85,1.85,0.0,0.00,0.00,nc6aOD1n,0.386100,0.684932,0.540541,0.540541,0.071032,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,103,107,2.88,1.68,0.000,0.000,0.394583,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11395,2024-04-11,Romênia Divisão A,12:00,SCM Craiova,Laguna Bucuresti,1.05,8.65,149.5,1.85,1.85,0.0,0.00,0.00,fqox4nZI,0.952381,0.115607,0.540541,0.540541,0.067988,133.246,53.826279,0.403962,3.0,0.000000,0.000000,91.52,1.634,0.633979,0.387992,62.0,550.492,177.033206,0.321591,0.0,0.000000,NaN,748.60,7.988,3.476244,0.435183,-64.0,88,76,1.04,9.85,109.712,903.608,1.108044,0.000000,NaN,0.21,0.042,1.190476,0.751364,0.5,-0.251364,-5.00,-1.000,-7.650000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11458,2024-04-11,Austrália Nbl1 Sul,05:00,Frankston F,Diamond Valley F,1.48,2.53,147.5,1.88,1.79,0.0,0.00,0.00,tO5eGazA,0.675676,0.395257,0.531915,0.558659,0.070933,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,66,53,1.51,3.69,0.000,0.000,0.370305,0.034681,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11459,2024-04-11,Austrália Nbl1 Sul,07:00,Ringwood,Knox,2.84,1.39,173.5,1.80,1.95,0.0,0.00,0.00,rZClvifG,0.352113,0.719424,0.555556,0.512821,0.071537,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11428,20:00,Argentina Liga A,Penarol,Obera TC,1.54,Back Home
11437,20:30,Eua Nba,Boston Celtics,New York Knicks,1.66,Back Home
